In [ ]:
from typing import Callable, Union, Dict, List, Optional
import random
from abc import abstractproperty, ABC
import numpy as np
from pathlib import Path
import tempfile
import uuid

class Example(ABC):
    def __init__(self, features, label):
        self.features = Features(features)
        self.label = label

    @abstractproperty
    def txt(self) -> List[str]:
        ...

    @abstractproperty
    def dsjson(self) -> str:
        ...

class Regression(Example):
    @property
    def txt(self) -> List[str]:
        return f'{self.label} |{self.features.txt}'

    @property
    def dsjson(self) -> str:
        raise NotImplementedError()


def feat_2_text(features: Dict):
    def _kv_2_str(k, v):
        if isinstance(v, str):
            return f'{k}={v}'
        elif isinstance(v, list):
            raise NotImplementedError()
        else:
            return f'{k}:{v}'
    return ' '.join(map(lambda kv: _kv_2_str(kv[0], kv[1]), features.items()))

class Features:
    def __init__(self, impl):
        self.impl = impl

    @staticmethod
    def _flat(d, key, result: Optional[dict] = None):
        result = result or {}
        if isinstance(d, Dict):
            for k, v in d.items():
                if isinstance(v, Dict):
                    result = Features._flat(v, k, result)
                else:
                    if key not in result:
                        result[key] = {}
                    result[key][k] = v
        if isinstance(d, List):
            raise NotImplementedError()
        return result    

    @property
    def txt(self):
        return '|'.join(f'{k} {feat_2_text(v)} 'for k,v in self._flat(self.impl, '').items())


class ExampleGen:
    def __init__(self, feature_gen, label_gen, example_type):
        self._fg = FeatureGen(feature_gen)
        self._lg = label_gen
        self._et = example_type

    def get(self, i):
        features = self._fg.get(i)
        return self._et(features, self._lg(features, i))


class FeatureGen:
    def __init__(self, impl):
        self._impl = impl 

    def _eval(d: Union[Dict, List], i):
        if isinstance(d, Dict):
            result = {}
            for k, v in d.items():
                if isinstance(v, Dict) or isinstance(v, List):
                    result[k] = FeatureGen._eval(v, i)
                elif isinstance(v, Callable):
                    result[k] = v(i)
                else:
                    result[k] = v
            return result
        if isinstance(d, List):
            result = []
            for v in d:
                if isinstance(v, Dict) or isinstance(v, List):
                    result.append(FeatureGen._eval(v, i))
                elif isinstance(v, Callable):
                    result.append(v(i))
                else:
                    result.append(v)
            return result
        raise ValueError(f'Unsupported type for the object: {type(d)}')         

    def get(self, i):
        return FeatureGen._eval(self._impl, i)


def _get_temp_path():
    return Path(tempfile.gettempdir()) / str(uuid.uuid4())


class Simulation(list):
    def __init__(self, example_gen, count, seed = 0):
        random.seed(seed)
        np.random.seed(seed)
        super().__init__([example_gen.get(i) for i in range(count)])

    @property
    def txt(self):
        return map(lambda ex: ex.txt, self)

    def write_txt(self, path=None):
        path = path or _get_temp_path()
        with open(path, 'w') as f:
            for ex in self.txt:
                f.write(f'{ex}\n')
        return path


In [ ]:
from vw_executor.vw import Vw
from vw_executor.vw_opts import Grid

vw = Vw('.cache', 'vw')

In [ ]:
sim = Simulation(
    ExampleGen(
        {
            'a': {
                'x': lambda _: random.random() * 10
            }
        },
        lambda obj, _: obj['a']['x']**2 + 1,
        Regression
    ),
    10000
)
path = sim.write_txt()

In [ ]:
import pandas as pd
results = vw.train(
    path,
    pd.DataFrame(Grid({
        '#args': ['--coin', '--freegrad', '--learning_rate 0.1'],
        '#interactions': ['', '-q aa']
    })))
results

In [ ]:
result